In [18]:
import numpy as np
import pickle
import os

from api_neurotask import *

import matplotlib.pyplot as plt
import seaborn as sns
from plot_utils import adjust_spines

## Datasets

In [24]:
dataset_names = []
for root, dirs, files in os.walk('./data'):
    for file in files:
        if file.endswith('.parquet'):
            dataset_names.append(root + '/' + file)

Choose dataset:

In [33]:
dataset=dataset_names[0]
dataset

'./data/4_1_MaXuan_Key.parquet'

Load dataset

In [41]:
df, bin = load_and_filter_parquet(dataset, ['A', 'I','F'])

Data loaded from ./data/4_1_MaXuan_Key.parquet with bin size of 1 ms
Events columns: ['EventGo_cue', 'EventMovement_start', 'EventTarget_Onset']
Covariates columns: ['force_x', 'force_y', 'target_dir']


In [95]:
neurons = [neuron for neuron in df.columns if neuron.startswith('Neuron')]

### Run GPFA on all sessions and animals in this dataset:

We run gpfa, based on core implementation in [elephant](https://elephant.readthedocs.io/en/latest/reference/gpfa.html), but without using Neo spike train preprocessing.

In [67]:
from gpfa_utils import dataframe_to_spike_trains
from elephant.gpfa import gpfa_core, gpfa_util

Hyperparameters: 

In [59]:
latent_dimensionality=6
min_var_frac=0.01
min_var_frac_explanation="""fraction of overall data variance for each observed dimension to set as
        the private variance floor.  This is used to combat Heywood cases,
        where ML parameter learning returns one or more zero private variances.
        Default: 0.01
        (See Martin & McDonald, Psychometrika, Dec 1975.)"""

tau_init=100.0 # ms # GP timescale initialization in msec
eps_init=1.0e-3 # GP noise variance initialization
em_tol=1.0e-8 # stopping criterion for EM
em_max_iters=500 # max EM iterations
freq_ll=5 # every freq_ll steps in EM likelihood is computed
verbose=False # feedback or not

We want bin sizes at least 10 ms big in this analysis.

In [53]:
new_bin=10.0

In [ ]:
for animal in df['animal'].unique():
    print('Animal id: ', animal)
    print('---------')
    for session in df[df['animal']==animal]['session'].unique():
        print('Session id: ', session)
        df_gpfa = df[(df['animal']==animal)&(df['session']==session)]
        bin_width=bin
        
        if bin < 10.0:
            df_gpfa = rebin(df_gpfa, prev_bin_size = bin, new_bin_size = new_bin)
            bin_width = new_bin
        
        seqs = dataframe_to_spike_trains(df_gpfa, neurons)
        
        # Check if training data covariance is full rank
        y_all = np.hstack(seqs["y"])
        y_dim = y_all.shape[0]

        if np.linalg.matrix_rank(np.cov(y_all)) < y_dim:
            errmesg = (
                "Observation covariance matrix is rank deficient.\n"
                "Possible causes: "
                "repeated units, not enough observations."
            )
            raise ValueError(errmesg)
        
        if verbose:
            print("Number of training trials: {}".format(len(seqs)))
            print("Latent space dimensionality: {}".format(latent_dimensionality))
            print(
                "Observation dimensionality: {}".format(
                    has_spikes_bool.sum()
                )
            )
        
        
        # Fit
        params_estimated, fit_info = gpfa_core.fit(
            seqs_train=seqs,
            x_dim=latent_dimensionality,
            bin_width=bin_width,
            min_var_frac=min_var_frac,
            em_max_iters=em_max_iters,
            em_tol=em_tol,
            tau_init=tau_init,
            eps_init=eps_init,
            freq_ll=freq_ll,
            verbose=verbose
        )
        
        # Transform
        transform_info = dict()
        returned_data=['latent_variable', 'VsmGP']
        
        seqs, ll = gpfa_core.exact_inference_with_ll(
            seqs, params_estimated, get_ll=True
        )
        transform_info["log_likelihood"] = ll
        transform_info["num_bins"] = seqs["T"]
        
        # Orthonormalize columns in C, update latents
        Corth, seqs = gpfa_core.orthonormalize(params_estimated, seqs)
        
        transform_info["Corth"] = Corth
        if len(returned_data) == 1:
            gpfa_val_result = seqs[returned_data[0]]
        gpfa_val_result =  {x: seqs[x] for x in returned_data}
        
        with open('save_pickles/{}_animal_{}_session_{}_latent_dim_{}.pickle'.format(
            dataset.split('/')[-1].split('.')[0],
            animal_id,
            session_id,
            latent_dimensionality
        ), 'wb') as f:
            pickle.dump({
                'params':params_estimated,
                'latents':gpfa_val_result
            }, f)

Animal id:  1
---------
Session id:  1
Initializing parameters using factor analysis...

Fitting GPFA model...
